# G2ConfigMgr Reference

## Prepare environment

In [ ]:
import os
import sys
import json

# For RenderJSON

import uuid
from IPython.display import display_javascript, display_html, display

### Helper class for JSON rendering

A class for pretty-printing JSON.
Not required by Senzing, 
but helps visualize JSON.

In [ ]:
class RenderJSON():
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, bytearray):
            self.json_str = json_data.decode()
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height:100%; width:100%; background-color: LightCyan"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### System path

Update system path.

In [ ]:
python_path = "{0}/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables used for G2Config and G2ConfigMgr.

In [ ]:
%run senzing-init-config.ipynb

In [ ]:
%store -r senzing_config_json

In [ ]:
print(senzing_config_json)

### Create configuration object

The G2Config API is used to create a new JSON configuration from the default template.
For more information, 
view the [senzing-G2Config-sdk-api-specification.ipynb](senzing-G2Config-sdk-api-specification.ipynb) notebook.

In [ ]:
from senzing import G2Config, G2Exception

g2_config = G2Config()
try:
    g2_config.init(module_name, senzing_config_json, verbose_logging)

    # Create configuration from template file.

    config_handle = g2_config.create()

    # Save JSON to string variable.

    response_bytearray = bytearray()
    g2_config.save(config_handle, response_bytearray)
    senzing_model_config_json = response_bytearray.decode()

except G2Exception as err:
    print(g2_config.getLastException())

print(response_bytearray.decode())

## G2ConfigMgr

The G2ConfigMgr API is used to add specific JSON configurations to the database,
so that they may be shared across remote systems.
Such configurations are added to the database, and a configuration ID is created for each of them.
The system may then be configured with a specific configuratin ID that points to one of those configurations.
That configuration ID will then be the shared global config.

In [ ]:
from senzing import G2ConfigMgr

### G2ConfigMgr Initialization

To start using G2ConfigMgr, you must first create and initialize an instance of the config manager.
This should be done once per process.

Create a new instance of the config manager and assign it to a variable.
Then, call the appropriate initialization method to initialize the config manager.

###### Parameters

- **module_name:** (str) A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **senzing_config_json:** (JSON str) A JSON document containing system parameters.
- **verbose_logging:** (bool) A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)

In [ ]:
g2_configuration_manager = G2ConfigMgr()
try:
    g2_configuration_manager.init(
        module_name,
        senzing_config_json,
        verbose_logging)

except G2Exception as err:
    print(g2_configuration_manager.getLastException())

### addConfig

Use `addConfig()` to add a configuration JSON document to the data repository.

###### Parameters

- **senzing_model_config_json:** (JSON str) The configuration JSON document.
- **config_comment:** (str) A free-form string of comments describing the configuration document.
- **config_id_bytearray:** (bytearray) The returned configID for the new config document registered in the data store.

In [ ]:
config_comment = "Configuration added from G2SetupConfig."
config_id_bytearray = bytearray()
try:
    g2_configuration_manager.addConfig(
        senzing_model_config_json,
        config_comment,
        config_id_bytearray)

except G2Exception as err:
    print(g2_configuration_manager.getLastException())
print("Config ID: {0}".format(config_id_bytearray.decode()))

###  getConfigList

Use `getConfigList()` to retrieve a list of the configuration JSON documents contained in the data repository.
###### Parameters

- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
response_bytearray = bytearray()
try:
    g2_configuration_manager.getConfigList(response_bytearray)

except G2Exception as err:
    print(g2_configuration_manager.getLastException())

RenderJSON(response_bytearray)

### getConfig

Use `getConfig()` to retrieve a specific configuration JSON document from the data repository. 

###### Parameters

- **config_id_bytearray:** (bytearray) The configID for the config document that you wish to retrieve.

- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
response_bytearray = bytearray()
try:
    g2_configuration_manager.getConfig(
        config_id_bytearray,
        response_bytearray)

except G2Exception as err:
    print(g2_configuration_manager.getLastException())

RenderJSON(response_bytearray)

### setDefaultConfigID

Use `setDefaultConfigID()` to set the default configuration JSON document in the data repository. 

###### Parameters

- **config_id_bytearray:** (bytearray or int) The configuration ID for a configuration JSON document previously added to the database.

In [ ]:
try:
    g2_configuration_manager.setDefaultConfigID(config_id_bytearray)

except G2Exception as err:
    print(g2_configuration_manager.getLastException())

### getDefaultConfigID

Use `getDefaultConfigID()` to retrieve a specific configuration JSON document from the data repository.

###### Parameters

- **config_id_bytearray:** (bytearray) Returns the configID for the current default configuration, or 0 if none is set.

In [ ]:
try:
    g2_configuration_manager.getDefaultConfigID(config_id_bytearray)

except G2Exception as err:
    print(g2_configuration_manager.getLastException())
print("Configuration ID: {0}".format(config_id_bytearray.decode()))

## Cleanup

### getLastException

In [ ]:
print(g2_config.getLastException())

### clearLastException

In [ ]:
try:
    g2_config.clearLastException()

except G2Exception as err:
    print(g2_config.getLastException())

### close

- **config_handle:** (int) variable containg the the number of the used config

In [ ]:
try:
    g2_config.close(config_handle)

except G2Exception as err:
    print(g2_config.getLastException())

### destroy

In [ ]:
try:
    g2_config.destroy()

except G2Exception as err:
    print(g2_config.getLastException())